# Resumo relatório final

## Modelo Memory Polynomial (MP)

O modelo MP é descrito pela seguinte equação: 

$$\tilde{out}_n = \sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{coefs}_{mp}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$$

Onde $\tilde{out}$ e $\tilde{in}$ são vetores de valores complexos.



## Cálculo dos coeficientes

O cálculo dos coeficientes é feito como descrito na equação a seguir:

$$ \tilde{coefs}_{mp} = \frac{\tilde{out}_n}  {\sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}} $$

Essa operação pode ser realizada utilizando a abordagem com matrizes complexas. 

In [1]:
import scipy.io as spio
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt


mat = spio.loadmat('data/data_LDMOS.mat')
in_ext = mat['in_extraction']
out_ext = mat['out_extraction']
in_val = mat['in_validation']
out_val = mat['out_validation']

P = 5
M = 2
n = 2


def x_mp(entrada, M, P):
    modulo_entrada = np.absolute(entrada)
    X_MP = np.zeros((len(entrada),P*(M+1)),dtype=complex)
    for r in range(M+1,len(entrada)):
        for m in range(M+1):
            for p in range(1,P+1):
                c = ((m*P)-1)+p
                X_MP[r,c] = entrada[r-m,0]*((modulo_entrada[r-m,0])**(p-1))
    return X_MP

X_ext = x_mp(in_ext,M,P)
X_ext2 = X_ext[M+3:len(X_ext)-(M+3),:]
out_ext2 = out_ext[M+3:len(out_ext)-(M+3),:]
coefs = np.linalg.lstsq(X_ext2,out_ext2,rcond=-1)
coefs = coefs[0]
print(coefs)

[[ 1.2622929 -5.32196973e-03j]
 [-1.69747655+5.09301593e-02j]
 [ 3.45340987+7.74461073e-01j]
 [-3.30856233-2.26594244e+00j]
 [ 0.91061063+1.31286342e+00j]
 [-0.05662761+4.03202604e-04j]
 [ 0.32810271-1.40232176e-01j]
 [ 0.78543012+1.77217523e-01j]
 [-1.80250055-6.56437588e-02j]
 [ 0.95415195-4.09195545e-02j]
 [-0.04889592+8.81045717e-03j]
 [ 0.1645401 -3.24559864e-02j]
 [-0.83237918+1.70448069e-01j]
 [ 1.15547792-2.82329965e-01j]
 [-0.53874228+1.69059491e-01j]]


Os coeficientes como estão calculados são dispostos da seguinte forma: 

$$coef_{mp} = \begin{bmatrix} b_{01} \\ b_{02} \\ b_{03} \\ b_{11} \\ b_{12} \\ b_{13} \end{bmatrix} $$

Obs.: considerando M=1 e P=3


## Validação usando matrizes complexas:

Seguindo a equação:

$$\tilde{out}_n = \sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{coefs}_{mp}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$$

Para calcularmos a saída estimada para validação precisamos apenas multiplicar os coeficientes calculados anteriormente por $\sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$


In [2]:
X_val = x_mp(in_val,M,P)
X_val2 = X_val[M+3:len(X_val)-(M+3),:]
out_calc_mat_cmplx = X_val2@coefs
print(out_calc_mat_cmplx)

[[0.17080977-0.45086617j]
 [0.09153475-0.46860405j]
 [0.00505675-0.46001652j]
 ...
 [0.13832978+0.07226911j]
 [0.1237369 +0.11137137j]
 [0.11426619+0.14626386j]]


### NMSE 

A métrica utilizada comparar o desempenho do modelo em cada abordagem será o NMSE, que é descrito pela seguinte equação:

$$ NMSE = 10log_{10}\frac{\sum^{N}_{n=1}|e_n|^2}{\sum^{N}_{n=1}|out_n|^2}$$

onde $out_n$ é o valor da amostra do sinal de saída no instante n, $e_n$ é o valor calculado
do erro entre as amostras do sinal de saída real e do sinal de saída simulado pelo modelo,
ou seja, $out_n − outcalc_n$ e N representa a quantidade total de amostras disponíveis.

In [3]:
def NMSE(previsao, validacao):
    erro = validacao - previsao
    erro = np.sum(np.absolute(erro)**2)
    val = np.sum(np.absolute(validacao)**2)
    res = 10*np.log10(erro/val)
    return res
out_val2 = out_val[M+3:len(X_val)-(M+3),:]
NMSE(out_calc_mat_cmplx,out_val2)

-38.44451617229987

## Validação usando LUTS:

In [13]:
def in_abs_lut(entrada, M, P):
    modulo_entrada = np.absolute(entrada)
    in_abs = np.zeros((len(entrada),P*(M+1)),dtype=complex)
    for r in range(M+1,len(entrada)):
        for m in range(0,M+1,1):
            for p in range(1,P+1):
                c = ((m*P)-1)+p
                in_abs[r,c] = (modulo_entrada[r-m,0])**(p-1)
    return in_abs


def lut(in_abs,M,P,n,coef):
    xpol2 = in_abs[:,0:P-1]@coef[0:P-1]
    for i in range(1,M+1):
        xpol = in_abs[:,i*P:((i*P)*(i+1))-1]@coefs[i*P:((i*P)*(i+1))-1]
        xpol2 = np.append(xpol2,xpol,axis=1)
    xpol2 = xpol2[M+1:-1,:]

    Q = 2**n
    z=np.linspace(0,1,Q)

    z = [int(round(a*(len(xpol2)-1))) for a in z]  
    
    lut = np.zeros((Q,2*(M+1)))
    c1 = -1
    for c in range(0,2*(M+1),2):
        c1+=1
        for r in range(Q):
            lut[r,c] = np.real(xpol2[z[r],c1])
            lut[r,c+1] = np.imag(xpol2[z[r],c1])
    return lut

n=2
in_lut = in_abs_lut(in_val,M,P)
lut = lut(in_lut, M, P, n,coefs)
print('LUT: ',lut)

LUT:  [[ 0.91073799 -0.01765733  0.07873419 -0.03008736 -0.0456104   0.00786506]
 [ 0.96916181  0.01733429  0.0608937  -0.02700679 -0.04379537  0.00759249]
 [ 0.94204693  0.00633895  0.07895665 -0.03013091 -0.04884979  0.0083399 ]
 [ 1.07380018  0.01215526  0.02053723 -0.02007679 -0.04101311  0.00716631]]


In [14]:
def interpolacao(entrada,lut,M,n):
    Q = 2**n
    z = np.linspace(0,1,Q) 
    abs_in = np.abs(entrada)
    abs_in = abs_in[M+1:-1,:]
    inter = np.interp(abs_in,z,lut[:,0])
    for c in range(1,2*(M+1)):
        inter2 = np.interp(abs_in,z,lut[:,c])
        inter = np.append(inter,inter2,axis=1)
    
    real = np.zeros((len(inter),1))
    imag = np.zeros((len(inter),1))
    for c in range(0,2*(M+1),2):
        real = np.add(real,inter[:,c].reshape(len(inter),1))
        imag = np.add(imag,inter[:,c+1].reshape(len(inter),1))
    interpol = real + (imag * 1j)
    return interpol

inter = interpolacao(in_val,lut,M,n)
in_val_lut = in_val[M+1:-1,:]
out_val_lut = out_val[M+1:-1,:]
out_lut = inter*in_val_lut
NMSE_lut = NMSE(out_lut,out_val_lut)
print(NMSE_lut)

-25.98226246602085


## Validação usando matrizes reais:

Usando a equação:


$$\begin{bmatrix} a + jb & c + jd \end{bmatrix} * \begin{bmatrix} e + jf \\ g + jh \end{bmatrix} = \begin{bmatrix} a & b & c & d \end{bmatrix} * \begin{bmatrix} e & f \\ -f & e \\ g & h \\ -h & g \end{bmatrix}$$

Na etapa de multiplicação pelos coeficientes.

In [15]:
def in_abs_real(in_val,P):
    res = np.zeros((len(in_val),P))
    for c in range(1,P+1,1):
        for r in range(len(in_val)):
            res[r,c-1] = (np.abs(in_val[r]))**(c-1)
    return res


def x_mp_real(in_val,M):
    res0 = np.zeros((len(in_val),M+1),dtype=complex)
    for c in range(M+1):
        for r in range(len(in_val)-c):
            res0[r+c,c] = in_val[r,0]

    res = np.zeros((len(in_val),2*(M+1)))   
    c2=0   
    for c in range(0,2*(M+1),2):
            res[:,c] = np.real(res0[:,c2])
            res[:,c+1] = np.imag(res0[:,c2])
            c2+=1
    return res


in_real = in_abs_real(in_val,P)
x_real = x_mp_real(in_val,M)

In [16]:
def mult1(in_real,x_real,M,P):
    res = np.zeros((len(in_real),2*P*(M+1)))
    zero = np.zeros((1,in_real.shape[1]))
    c=-2
    for c1 in range(0,2*(M+1),2):
        if c1 != 0:
            in_real = np.append(zero,in_real,axis=0)
            in_real = in_real[0:in_real.shape[0]-1,:]
        for c2 in range(P):
            c+=2
            res[:,c] = x_real[:,c1]*in_real[:,c2] 
            res[:,c+1] = x_real[:,c1+1]*in_real[:,c2]
    res = res[0:len(res)-M,:]
    return res


def coef_real(coef):
    res0 = np.zeros((len(coef),2))
    for r in range(len(coef)):
        res0[r,0] = np.real(coef[r])
        res0[r,1] = np.imag(coef[r])
    res = np.zeros((2*len(coef),2))
    r2=0
    for r in range(len(res0)):
        res[r2,0]=res0[r,0]
        res[r2,1]=res0[r,1]
        res[r2+1,0]=-res0[r,1]
        res[r2+1,1]=res0[r,0]
        r2+=2
    return res

mult1_re = mult1(in_real,x_real,M,P)
coef_re = coef_real(coefs)
mult2_re = mult1_re@coef_re
out_mat_re = mult2_re[:,0]+(mult2_re[:,1]*1j)
out_mat_re = out_mat_re.reshape(len(out_mat_re),1)
print(out_mat_re)

[[0.20991968+0.01912632j]
 [0.25803087-0.10895927j]
 [0.2734164 -0.22573549j]
 ...
 [0.10894703+0.17060906j]
 [0.1057725 +0.1783943j ]
 [0.10256454+0.16411706j]]


In [17]:
out_val_re = out_val[0:len(out_val)-M,:]
NMSE(out_mat_re,out_val_re)

-38.43873579288449